# setup

In [1]:
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-c5kk6dth
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-c5kk6dth
  Resolved https://github.com/openai/whisper.git to commit ad3250a846fe7553a25064a2dc593e492dadf040
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!nvidia-smi

Sat Mar 11 06:30:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import glob
import os
import pandas as pd
import whisper

In [4]:
gdrive_home_path = "/content/drive"
from google.colab import drive
drive.mount(gdrive_home_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
DIR_NAME = "uraradi_transcript"

work_dir = os.path.join(gdrive_home_path, "MyDrive", DIR_NAME)
work_sounds_dir = os.path.join(work_dir, "sounds")
work_transcripts_dir = os.path.join(work_dir, "transcripts")

# identify existing mp3/transcript

In [6]:
mp3_list = []
for sounds_file in glob.glob(os.path.join(work_sounds_dir, "*.mp3")):
    filename = sounds_file.split("/")[-1].split(".")[0]
    mp3_list.append(filename)

csv_list = []
for transcripts_file in glob.glob(os.path.join(work_transcripts_dir, "*.csv")):
    filename = transcripts_file.split("/")[-1].split(".")[0]
    csv_list.append(filename)

# identify target radio

In [7]:
playlist = pd.read_csv(os.path.join(work_dir, "playlist_裏ラジオウルナイト.csv"))
playlist["flag"] = playlist["date"].apply(lambda date: (date in mp3_list)&(date not in csv_list))
target_radios = playlist[playlist["flag"]==True][["url", "date"]].reset_index(drop=True).copy()
target_radios

,url,date
0,https://www.youtube.com/watch?v=aZ6hRj6jPFs,2022-07-08


# to transcript

In [8]:
model = whisper.load_model("large")

In [ ]:
for _, radio in target_radios.iterrows():
    print(f"{radio.date} transcript start.")
    mp3_filename = radio["date"] + ".mp3"
    result = model.transcribe(os.path.join(work_sounds_dir, mp3_filename), language="ja", verbose=True)
    transcript_column = ["start_s", "end_s", "text"]
    transcript_list = []
    for segment in result["segments"]:
        start_s = int(segment["start"])
        end_s = int(segment["end"])
        row = [start_s, end_s, segment["text"]]
        transcript_list.append(row)
    transcript = pd.DataFrame(transcript_list, columns=transcript_column)
    csv_filename = radio["date"] + ".csv"
    transcript.to_csv(os.path.join(work_transcripts_dir, csv_filename), index=False)
    os.remove(os.path.join(work_sounds_dir, mp3_filename))
    print(f"{radio.date} transcript finish.")